### Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
from plotters import Plotter
import pandas as pd
from tqdm.notebook import tqdm
from scipy import stats
import seaborn as sns
import subjects
from stats_utils import get_bootstrap_prob, get_bootstrap_prob_paired
from stats_utils import bootstrap_resample

### Ripple rate

In [ ]:
# Uncomment if you want to re-run the bootstrapping

from stats_utils import bootstrap_resample, resample

rpl_rate = subjects.GroupData().ripple_rate.drop(["name"], axis=1)
func = lambda df: df.groupby(["grp", "zt"], sort=False).mean().reset_index()
rpl_df = bootstrap_resample(
    rpl_rate, level="session", n_iter=10000, n_jobs=6, apply=func
).drop(columns="session")

subjects.GroupData().save(rpl_df, "ripple_rate_bootstrap_session")

In [ ]:
rpl_df = subjects.GroupData().ripple_rate_bootstrap_session
fig = plotting.Fig(4, 5)
ax = fig.subplot(fig.gs[1])
plotter = Plotter(data=rpl_df, x="zt", y="rate", hue="grp", hue_order=["NSD", "SD"])
plotter.violinplot_sd(split=True, palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_across=get_bootstrap_prob,
    stat_within=get_bootstrap_prob_paired,
    alpha_within=0.025,
    alpha_across=0.025,
)

### Ripple properties

In [ ]:
## Uncomment if you want to re-run the bootstrapping
from stats_utils import bootstrap_resample

rpl_features = subjects.GroupData().ripple_features
func = lambda df: df.groupby(["grp", "zt"], sort=False).mean().reset_index()
rpl_df = bootstrap_resample(
    rpl_features, level="both", n_iter=10000, n_jobs=8, apply=func
)

subjects.GroupData().save(rpl_df, "ripple_bootstrap_session_ripples")

In [ ]:
rpl_df = subjects.GroupData().ripple_bootstrap_session_ripples
y_labels = [
    "Ripple frequency (Hz)",
    "Sharp wave amplitude \n(Normalized to PRE)",
    "Ripple power (zscore) \n(Normalized to PRE)",
]

fig = subjects.SdFig().fig1

for i, param in enumerate(["peak_freq", "sw_amp", "ripple_power"]):
    ax = fig.subplot(fig.gs[1, i])
    plotter = Plotter(data=rpl_df, x="zt", y=param, hue="grp", ax=ax)
    plotter.violinplot_sd(split=False, palette=subjects.colors_sd(1)).stat_anot_sd(
        stat_within=get_bootstrap_prob_paired,
        stat_across=get_bootstrap_prob,
        alpha_within=0.025,
        alpha_across=0.025,
        fontsize=5,
    )
    ax.legend("", frameon=False)
    ax.set_xlabel("")
    ax.set_ylabel(y_labels[i])

fig.savefig(subjects.figpath_sd / "ripple_bootstrap_test")

### (1h block) Ripple properties

In [5]:
## Uncomment if you want to re-run the bootstrapping
from stats_utils import bootstrap_resample

rpl_features = subjects.GroupData().ripple_features_1h_blocks
func = lambda df: df.groupby(["grp", "zt"], sort=False).mean().reset_index()
rpl_df = bootstrap_resample(
    rpl_features, level="both", n_iter=10000, n_jobs=8, apply=func
)

subjects.GroupData().save(rpl_df, "ripple_1h_blocks_bootstrap_session_ripples")

ripple_1h_blocks_bootstrap_session_ripples saved


In [ ]:
rpl_df = subjects.GroupData().ripple_1h_blocks_bootstrap_session_ripples
y_labels = [
    "Ripple frequency (Hz)",
    "Sharp wave amplitude \n(Normalized to PRE)",
    "Ripple power (zscore) \n(Normalized to PRE)",
]

fig = subjects.SdFig().fig1

for i, param in enumerate(["peak_freq", "sw_amp", "ripple_power"]):
    ax = fig.subplot(fig.gs[1, i])
    plotter = Plotter(data=rpl_df, x="zt", y=param, hue="grp", ax=ax)
    plotter.violinplot_sd(split=False, palette=subjects.colors_sd(1)).stat_anot_sd(
        stat_within=get_bootstrap_prob_paired,
        stat_across=get_bootstrap_prob,
        alpha_within=0.025,
        alpha_across=0.025,
        fontsize=5,
    )
    ax.legend("", frameon=False)
    ax.set_xlabel("")
    ax.set_ylabel(y_labels[i])

# fig.savefig(subjects.figpath_sd / "ripple_bootstrap_test")

### Normalized ripple properties

In [4]:
## Uncomment if you want to re-run the bootstrapping
from stats_utils import bootstrap_resample

rpl_features = subjects.GroupData().ripple_features_normalized
func = lambda df: df.groupby(["grp", "zt"], sort=False).mean().reset_index()
rpl_df = bootstrap_resample(
    rpl_features, level="both", n_iter=10000, n_jobs=8, apply=func
)

subjects.GroupData().save(rpl_df, "ripple_normalized_bootstrap_session_ripples")

100%|██████████| 10000/10000 [06:26<00:00, 25.87it/s]


ripple_normalized_bootstrap_session_ripples saved


### Get bootstrap means for ripple properties statewise

In [ ]:
# Uncomment if you want to re-run the bootstrapping

rpl_features = subjects.GroupData().ripple_features_brainstate
# rpl_features = rpl_features[
#     ~((rpl_features.zt == "2.5-5") & (rpl_features.grp == "SD"))
# ]

func = lambda df: df.groupby(["zt"], sort=False).mean(numeric_only=True).reset_index()

for state in ["WK", "NREM"]:
    rpl_df = bootstrap_resample(
        rpl_features[rpl_features.brainstate == state],
        level="both",
        n_iter=10000,
        n_jobs=8,
        apply=func,
    )

    subjects.GroupData().save(rpl_df, f"ripple_{state}_bootstrap_session_ripples")

In [ ]:
df1 = subjects.GroupData().ripple_NREM_bootstrap_session_ripples
df1["state"] = "NREM"
df2 = subjects.GroupData().ripple_WK_bootstrap_session_ripples
df2["state"] = "WK"

df = pd.concat([df1, df2], ignore_index=True)


fig = plotting.Fig(5, 5)
for g, grp in enumerate(["NSD", "SD"]):
    # for i, df in enumerate([df1, df2]):
    ax = fig.subplot(fig.gs[g])
    plotter = Plotter(
        data=df[df.grp == grp],
        x="zt",
        y="peak_freq",
        hue="state",
        hue_order=["WK", "NREM"],
        ax=ax,
    )
    plotter.violinplot_sd(palette=None)
    ax.set_ylim([140, 190])

# fig.savefig(subjects.figpath_sd / "ripple_brainstate_sd")

### Bootstrap ripple rate trend (5 minutes window)

In [7]:
from ripple_utils import get_ripple_rate_trend

rpl_rate = subjects.GroupData().ripple_rate_post5h_trend

boot_df = bootstrap_resample(
    rpl_rate, level="session", n_iter=10000, n_jobs=6, apply=get_ripple_rate_trend
)

subjects.GroupData().save(boot_df, "ripple_rate_post5h_trend_bootstrap")

ripple_rate_post5h_trend_bootstrap saved
